# Safe First-Order Transion Systems

A verificação de propriedades lógicas de sistemas dinâmicos feita  em comportamentos ilimitados cinge-se, quase sempre, a propriedades de segurança da forma $\,\mathsf{G}\,\mathsf{P}\,$  em que $\,\mathsf{P}\,$ é uma propriedade de segurança expressa como um predicado nas variáveis de estado do sistema.
Como não é possível verificar fórmulas temporais genéricas, muitos modelos de sistemas dinâmicos incluem a propriedade $\,\mathsf{P}\,$ diretamente na formulação do próprio sistema e dispensam a lógica temporal. 
Mais exatamente é mais natural exprimir a propriedade de segurança $\,\mathsf{P}\,$ como a negação de uma condição de erro $\,\mathsf{E}\,$;  isto é, o sistema está num estado seguro se  não estiver num estado onde se verifique a condição de erro $\,\mathsf{E}\,$.


**Safe First Order Transition Systems (SFOTS)** são modelos de sistemas dinâmicos definidos por
$\:\:\:\Sigma\;\equiv\;\langle\,\mathsf{X}\,,\,\mathsf{next}\,,\,\mathsf{I}\,,\,\mathsf{T}\,,\,\mathsf{E}\,\rangle\:\:\:$
onde 
$\,\mathsf{X}\,$ é o conjunto das variáveis base que define o espaço de estados,
$\,\mathsf{next}\,$ é o operador que gera os vários “clones” dessas variáveis,
$\;\mathsf{I}\;$ é o predicado que determina os estados iniciais,
$\;\mathsf{T}\;$ é o predicado que define a relação de transição entre estados, e
$\;\mathsf{E}\;$ é o predicado que define uma condição de erro.

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import INT

import itertools 

Considere o seguinte programa anotado com uma pré-condição e com valor do *program counter*.
```Python
{ x >= 3 }
0: while (x>0):
1:     if x == 2:
2:        raise Error
3:     x = x-5
4: stop
```
Para modelar este programa como um SFOTS teremos o conjunto $\mathsf{X}$ de variáveis do estado dado pela lista `['x','pc']`, e definimos a função
`genState` que recebe a lista com o nome das variáveis do estado, uma etiqueta e um inteiro, e cria a i-ésima cópia das variáveis do estado para essa etiqueta. As variáveis lógicas começam sempre com o nome de base das variáveis dos estado, seguido do separador `!`.

In [3]:
def genState(vars,s,i):
    state = {}
    for v in vars:
        state[v] = Symbol(v+'!'+s+str(i),INT)
    return state

### Exercício 1

Defina as seguintes funções para completar a modelação deste programa:
- `init1` dado um estado do programa (um dicionário de variáveis), devolve um predicado do pySMT que testa se esse estado é um possível estado inicial do programa.
- `error1` dado um  estado do programa, devolve um predicado do pySMT que testa se esse estado é um possível estado de erro do programa.
- `trans1` que, dados dois estados do programa, devolve um predicado do pySMT que testa se é possível transitar do primeiro para o segundo estado

In [2]:
# completar

def init1(s):
    return And(GE(s['x'],Int(3)),
               Equals(s['pc'],Int(0)))
    
def error1(s):
    return Equals(s['pc'],Int(2))

def trans1(s,s_next):
    x, pc=s['x'],s['pc']
    x_n,pc_n=s_next['x'],s_next['pc']
    
    return Or(And(Equals(pc,Int(0)),GT(x,Int(0)),Equals(pc_n,Int(1)),Equals(x_n,x)),
              And(Equals(pc,Int(0)),GT(x,Int(0)),Equals(pc_n,Int(4)),Equals(x_n,x)),
              And(Equals(pc,Int(1)),GT(x,Int(2)),Equals(pc_n,Int(2)),Equals(x_n,x)),
              And(Equals(pc,Int(1)),GT(x,Int(2)),Equals(pc_n,Int(3)),Equals(x_n,x)),
              And(Equals(pc,Int(3)),Equals(pc_n,Int(0)),Equals(x_n,Minus(x,Int(5)))))
            

## Notação

O operador $\,\mathsf{next}\,$ está implícito na definição inicial de um FOTS quando se considera o conjunto de variáveis $\,\mathsf{X'}\,$ como “clones” das variáveis iniciais do sistema $\,\mathsf{X}\,$; de facto tem-se 
$\:\:\mathsf{X}'\,\equiv\,\mathsf{next}(\mathsf{X})$.

A mesma notação aplica-se a um qualquer predicado $\,\mathsf{P}\,$ que tenha $\,\mathsf{X}\,$ como o seu conjunto de variáveis livres:  representa-se por $\,\mathsf{P}'\,$ a fórmula nas variáveis livres $\,\mathsf{X}'\,$ que se obtém de $\,\mathsf{P}\,$ substituindo que cada variável $\,\mathsf{X}\,$ pela variável correspondente $\,\mathsf{X}'\,$. Temos portanto,
$\:\:\mathsf{P}'\;\equiv\;\mathsf{P}\,\{\mathsf{X}/\mathsf{next}(\mathsf{X})\}$.

A notação (usada nas aulas teóricas) segue as seguintes convenções:

- Todos os predicados unários são expressões que têm livres as variáveis $\;\mathsf{X}\;$.
- Todos os predicados binários são expressões que têm livres as variáveis $\;\mathsf{X}\;$ e $\;\mathsf{X}'\;$.
- Todo o predicado $n$-ário é uma expressão que tem livres variáveis $\,\mathsf{X}_0\,,\,\mathsf{X}_1\,\cdots\,\mathsf{X}_{n-1}\;$ com
 $\:\mathsf{X}_0\;\equiv\;\mathsf{X}\:$  e $\:\mathsf{X}_{i+1}\;\equiv\;\mathsf{next}(\mathsf{X}_i)\;$   para $\;i>0$
- Se $\;\mathsf{P}\;$ é um predicado $\,n$-ário então 
 $\mathsf{P}'\;\equiv\,\mathsf{P}\,\{\mathsf{X}_i/\mathsf{X}_{i+1}\;\text{para}\;i < n\!-\!1\;,\;\mathsf{X}_{n-1}/\mathsf{next}(\mathsf{X}_{n-1})\}$
   
Por conveniência, em todas estas sequências de variáveis $\,\mathsf{X}_0\,$ coincide com a variável $\,\mathsf{X}\,$ que ocorre na definição de $\,\Sigma\,$ e as restantes variáveis resultam de sucessivas invocações de $\,\mathsf{next}(\mathsf{X})$.
Nomeadamente a variável $\,\mathsf{X}_i\,$ resulta da $i$-ésima invocação de $\mathsf{next(X)}$.

Adicionalmente a função $\,\mathsf{top}\,$ que aplicada a um predicado $n$-ário $\,\mathsf{P}\,$ dá como resultado a última variável na sequência de variavéis que ocorrem em $\,\mathsf{P}$. 

Uma sequência de transições
$\,s_0 \stackrel{T}\longrightarrow s_1 \stackrel{T}\longrightarrow s_2 \cdots s_{n-1} \stackrel{T}\longrightarrow s_n\;$
pode ser definida indutivamente da  como potências da relação binária $\,\mathsf{T}\,$,
$\:\mathsf{T}^{n+1}\;\equiv\; \mathsf{T}\,\land\, (\mathsf{T}^{n})'\;$ para $\; n > 0$.
Para um predicado unário  $\;\mathsf{P}\;$, temos
$\:\mathsf{P}^0 \equiv \mathsf{True}\:\text{e}\; \mathsf{P}^{n+1}\equiv\mathsf{P}\,\land\,(\mathsf{P}^n)'\:$ para $\;n\geq 0$. Verifica-se que $\:(\mathsf{P}\land\mathsf{T})^n\equiv \mathsf{P}^n\,\land\,\mathsf{T}^n$.

### Exercício 2 
Escreva as fórmulas lógicas representadas nesta notação por $\mathsf{P}^{3}$ e $\mathsf{T}^{3}$. Assuma que $\mathsf{P}$ é um predicado unário e $\mathsf{T}$ um predicado binário.

$$
\begin{array}
\mathsf{P}^{3}\;\equiv\; ... \\
\mathsf{T}^{3}\;\equiv\; ...
\end{array}
$$

Seguindo esta notação, a fórmula 
$\;\mathsf{I}\,\land\,\mathsf{T}^n\,$ denota um traço finito com $\,n\,$ transições em $\Sigma\,$, $\,\mathsf{X}_0,\cdots,\mathsf{X}_n\,$, que descrevem estados acessíveis com $n$ ou menos transições. Inspirada nesta notação, a seguinte função `genTrace` gera um possível traço de execução com $n$ transições.

In [8]:
def genTrace(vars,init,trans,error,n):
    with Solver(name="z3") as s:
        
        X = [genState(vars,'X',i) for i in range(n+1)]   # cria n+1 estados (com etiqueta X)
        I = init(X[0])
        Tks = [ trans(X[i],X[i+1]) for i in range(n) ]
        
        if s.solve([I,And(Tks)]):      # testa se I /\ T^n  é satisfazível
            for i in range(n):
                print("Estado:",i)
                for v in X[i]:
                    print("          ",v,'=',s.get_value(X[i][v]))

### Exercício 3

Use esta função para testar a sua resposta ao exercício 1.

In [10]:
# completar
genTrace(['x','pc'],init1,trans1,error1,n=5)

Estado: 0
           x = 8
           pc = 0
Estado: 1
           x = 8
           pc = 1
Estado: 2
           x = 8
           pc = 3
Estado: 3
           x = 3
           pc = 0
Estado: 4
           x = 3
           pc = 1


## Segurança e acessibilidade


O algoritmo de verificação num SFOTS tem objetivos distintos da verificação usada nos FOTS.
Num SFOTS a propriedade de segurança (ou condição de erro) está contida na própria definição do sistema. A especificação do sistema determina o comportamento $\;\Sigma\;$ formado pelos traços mas também o subconjunto $\,\Sigma_U\subseteq \Sigma\;$ formado pelos traços que violam a condição de segurança.  Agora o algoritmo de verificação limita-se a  decidir se o conjunto $\;\Sigma_U\;$ é ou não vazio.


Num SFOTS $\:\Sigma\;\equiv\;\langle\,\,\mathsf{X}\,,\,\mathsf{next}\,,\,\mathsf{I}\,,\,\mathsf{T}\,,\,\mathsf{E}\,\rangle$  a  verificação deriva das noções de acessibilidade e insegurança. 
- Um estado $\,r\,$ é **acessível (“reachable”)** em $\,\Sigma\,$ quando $\,r\in\mathsf{I}\,$ ou quando existe uma transição $\;(s,r)\in \mathsf{T}\;$ em que $\,s\,$ é acessível em $\,\Sigma\,$.
- Um estado $\,u\,$ é **inseguro (“unsafe”)** em $\,\Sigma\,$ quando $\,u\in \mathsf{E}\,$ ou quando existe uma transição $\,(u,\upsilon)\in \mathsf{T}\,$ em que $\,\upsilon\,$ é inseguro em $\,\Sigma\,$.
- O SFOTS $\,\Sigma\,$ é **inseguro** se existe algum estado $\,s\,$ que seja simultaneamente acessível e inseguro. Em caso contrário o sistema $\,\Sigma\,$ é **seguro**.

A fórmula $\;\mathsf{R}_n\;\equiv\; \mathsf{I}\,\land\,\mathsf{T}^n\,$ denota um traço finito com $\,n\,$ transições em $\Sigma\,$, $\,\mathsf{X}_0,\cdots,\mathsf{X}_n\,$ que descrevem estados acessíveis com $n$ ou menos transições.  

Para descrever os estados inseguros, dá jeito  definir um novo SFOTS 
$\:\:\Sigma^\top\;\equiv\;\langle\,\mathsf{Y}\,,\,\mathsf{next}\,,\,\mathsf{E}\,,\,\mathsf{B}\,,\,\mathsf{I}\,\rangle\:\:$ onde $\,\mathsf{Y}\,$ é um “clone” da variável $\,\mathsf{X}\,$;  invocações sucessivas de $\,\mathsf{next}(\mathsf{Y})\,$  produzem variáveis; $\;\mathsf{E}$ é a nova condição inicial; $\;\mathsf{I}$  é a nova condição de erro; a relação de transição $\,\mathsf{B}\,$ é a relação inversa de $\,\mathsf{T}\,$. É fácil de ver que
> - Um estado $\,s\,$ é inseguro em $\,\Sigma\,$ se e só se $\,s\,$ é acessível em $\,\Sigma^\top\,$. 
> - Um estado $\,s\,$ é acessível em $\,\Sigma\,$ se e só se $\,s\,$ é inseguro em $\,\Sigma^\top\,$.


Este resultado permite-nos analisar as condições de segurança exclusivamente através das relações de acessibilidade em $\,\Sigma\,$ e $\,\Sigma^\top\,$.

### Exercício 4

Usando o sistema $\,\Sigma^\top$, escreva a fórmula (que convecionamos chamar $\:\mathsf{U}_m)\;$ que denota um traço finito com $\,m\,$ transições, cujos estados $\,\mathsf{Y}_0,\,\cdots\,,\mathsf{Y}_m\;$ são sempre inseguros. 
                                       
$$\mathsf{U}_m\;\equiv\; ... $$

### Exercício 5

Defina uma função de ordem superior `invert` que recebe a função python que codifica a relação de transição e devolve a relação e transição inversa.

In [11]:
# completar

def invert(trans):
    def b(u,v):
        return trans(v,u)
    return b

Para avaliar a segurança eventual do sistema $\;\Sigma\;$ é necessário determinar se nenhum estado é simultâneamente acessível e inseguro. Para isso tem que se avaliar se, para todo o $\,n,\,m\,$, a fórmula  $\:\:V_{n,m}\,\equiv\,\mathsf{R}_n\,\land\,(X_n=Y_m) \,\land\,\mathsf{U}_m\:\:$ é insatisfazível, onde $\;X_n = \mathsf{top}(\mathsf{R}_n)\:\text{e}\: Y_m = \mathsf{top}(\mathsf{U}_m)$.


## Um algoritmo de "model-checking" usando interpolantes e invariantes

Para provar que $\,\mathsf{V}_{n,m}\,$ é insatisfazíevel para todo $\,n,m\,$ vai-se usar uma estratégia similar ás provas por indução. Para tal precisamos de recorrer a interpolantes e a invariantes.

Um **interpolante** para o par $\;(A,B)\;$ de duas fórmula de 1ª ordem é definível quando $\,A\,$ e $\,B\,$ são mutuamente inconsistentes (isto é, $\;A\,\land\,B\;$ é insatisfazível) e, nesse caso, é uma fórmula de 1ª ordem $\;C\;$ tal que:
- $C\;$ só contém as variáveis comuns a $A$ e a $B$.
- Os pares $\;(A,\neg\,C)\;$ e $\;(C,B)\;$ são ambos mutuamente inconsistentes.

Na sua formulação mais simples, os invariantes são predicados cujo valor lógico se mantém após a execução da  transição $\;\mathsf{T}$.
Esta ideia pode ser generalizada para qualquer predicado  $\,n$-ário $\,\mathsf{F}\,$ nas variáveis $\;X_0,\cdots,X_n\;$ , com $\,n\geq 1\,$. 
Neste contexto o predicado $\,\mathsf{P}\,$ é **invariante** quando, sendo $\,X_n \equiv \mathsf{top}(\mathsf{F})\,$, se verifica que 
$\mathsf{P}\,\land\,\mathsf{F}\,\to\,\mathsf{P}\{\mathsf{X}/X_n\}\:$ é uma tautologia.

Se $\;\mathsf{P}\,,\,\mathsf{Q}\;$ são ambos predicado em $\,\mathsf{X}\,$,  diz-se que $\,\mathsf{P}\;$ é **invariante relativo a $\,\mathsf{Q}\;$ para a transição $\;\mathsf{T}\;$** quando 
$\:\mathsf{Q}\,\land\,\mathsf{P}\,\land\,\mathsf{T}\,\to\,\mathsf{P}'\:$
é uma tautologia.


O algoritmo de “model-checking” pretende verificar se para todo $\,n,m\,$ a fórmula $\,\mathsf{V}_{n,m}\,$ é insatisfazível.
O algoritmo derivado das noções de interpolante e invariante baseia-se no seguinte resultado

> Se existe um predicado unário $\,S\,$ que é invariante de $\;\mathsf{T}\,$ e, para algum par de índices $(n,m)\,$ verifica-se que  $\;\mathsf{R}_n(\overline{X}_n) \,\to\,S(X_n)\;$ e $\;\mathsf{U}_m(\overline{Y}_m)\,\to\,\neg\,S(Y_m)\;$  são tautologias , então $\;\mathsf{V}_{n',m'}\;$ é insatisfazível para todo $\;n'\ge n\;$ e $\;m'\ge m\;$.



### O algoritmo de "model-checking"

O algoritmo de “model-checking” manipula as fórmulas $\;\mathsf{R}_n\;\equiv\; \mathsf{I}\,\land\,\mathsf{T}^n\;$ e $\;\mathsf{U}_m\equiv\; \mathsf{E}\,\land\,\mathsf{B}^m\;$ fazendo crescer os índices $\;n,m\;$ de acordo  com as seguintes regras

----------
1. Inicia-se $n=0$, $\;\mathsf{R}_0 = \mathsf{I}\;$ e $\;\mathsf{U}_0 = \mathsf{E}$.


2.  No estado $\,(n,m)\,$  tem-se a certeza que em todos os estados anteriores não foi detectada nenhuma justificação para a insegurança do SFOTS. 
    Se $\;\mathsf{V}_{n,m}\equiv\;\mathsf{R}_n\land(X_n=Y_m)\land\mathsf{U}_m\;$ é satisfazível o sistema é inseguro e o algoritmo termina com a mensagem **unsafe**.


3. Se $\;\mathsf{V}_{n,m}\equiv\;\mathsf{R}_n\land(X_n=Y_m)\land \mathsf{U}_m\;$ for insatisfazível calcula-se  $\;{C}\;$ como  o interpolante do par $\,(\mathsf{R}_n\land(X_n=Y_m)\,,\,\mathsf{U}_m)\,$.
     Neste caso verificam-se as tautologias    $\mathsf{R}_n \to C(X_n)\;$  e $\;\mathsf{U}_m\,\to\,\neg\, C(Y_m)\;$.


4. Testa-se a condição $\;\mathsf{SAT}(C\land\mathsf{T}\land\neg\,C')=\emptyset\;$  para verificar se  $\,C\,$ é um invariante de $\;\mathsf{T}\,$; se for invariante  então, pelo resultado anterior, sabe-se que $\,\mathsf{V}_{n',m'}\;$ é insatisfazível para todo $\,n'\ge n\,$ e $\,m'\ge n\,$. O algoritmo termina com a mensagem **safe**.


5. Se $\,C\,$ não for invariante de $\,\mathsf{T}\,$ procura-se encontrar um majorante $\,S \supseteq C$ que verifique as condições do resultado referido: seja um invariante de $\,\mathsf{T}$   disjunto de $\,\mathsf{U}_m\,$.


6. Se for possível encontrar tal majorante $\,S\,$ então o algoritmo termina com a mensagem **safe**. Se não for possível encontrar o majorante pelo menos um dos índices $\,n,m\,$ é incrementado, os valores das fórmulas $\,\mathsf{R}_n\,,\,\mathsf{U}_m\,$ são actualizados e repete-se o processo a partir do passo 2.


#### Para encontrar um majorante $S$
A parte crítica é o passo 5. Várias estratégias são possíveis (veremos algumas mais tarde). 
Uma solução possível é um algoritmo iterativo que tenta encontrar um invariante $\,S\,$ pelos passos seguintes

1. $S\;$ é inicializado com $\,C(X_n)$
2. Faz-se $\,A \equiv S(X_n)\,\land\,\mathsf{T}(X_n,Y_m)\,$ e verifica-se se  $\,A\land U_m\;$ é insatisfazível. Se for satisfazível então não é possível encontrar o majorante e esta rotina termina sem sucesso.
3. Se $\,A \land U_m\;$ for insatisfazível calcula-se um novo interpolante $\,C(Y_m)\,$ deste par $(A,U_m)$.
4. Se $\,C(X_n)\,\to\,S\;$ for tautologia, o invariante pretendido está encontrado.
5. Se $\,C(X_n)\,\to\,S\;$ não é tautologia, actualiza-se $\,S\,$ com $\,S \,\lor\, C(X_n)\,$ e repete-se o processo a partir do passo (1).

Para auxiliar na implementação deste algoritmo, começamos por definir duas funções.
A função `rename` renomeia uma fórmula (sobre um estado) de acordo com um dado estado. 
A função `same` testa se dois estados são iguais.

In [12]:
def baseName(s):
    return ''.join(list(itertools.takewhile(lambda x: x!='!', s)))

def rename(form,state):
    vs = get_free_variables(form)
    pairs = [ (x,state[baseName(x.symbol_name())]) for x in vs ]
    return form.substitute(dict(pairs))

def same(state1,state2):
    return And([Equals(state1[x],state2[x]) for x in state1])

### Exercício 6

Complete a função de ordem superior `model-checking` que dada a lista de nomes das variáveis do sistema, um predicado que testa se um estado é inicial, um predicado que testa se um par de estados é uma transição válida, 
 um  predicado que testa se um estado é de erro, e dois números positivos N e M que são os limites máximos para os indices $n$ e $m$, implemente o algoritmo acima descrito com o auxílio de m SMT solver.

In [ ]:
# from pysmt.shortcuts import interpolate
def model_checking(vars,init,trans,error,N,M,max_maj):
    with Solver(name="z3") as s:
        
        # Criar todos os estados que poderão vir a ser necessários.
        X = [genState(vars,'X',i) for i in range(N+1)]
        Y = [genState(vars,'Y',i) for i in range(M+1)]
        
        B= invert(trans)
        # Estabelecer a ordem pela qual os pares (n,m) vão surgir. Por exemplo:
        order = sorted([(a,b) for a in range(1,N+1) for b in range(1,M+1)],key=lambda tup:tup[0]+tup[1]) 
        
        for (n,m) in order:
             
            # completar    
            R_parts=[init(X[0])]
            for i in range(n):
                R_parts.append(trans(X[i],X[i+1]))
            Rn=And(R_parts) if len (R_parts) > 1 else R_parts[0]
            U_parts = error(Y[0])
            for j in range(n):
                U_parts.append(B(Y[j],Y[j+1]))
                
            Um=And(U_parts) if len(U_parts)>1 else U_parts[0]
            
            Eq_top=same(X[n],Y[n])
            
            with Solver(name='z3') as s_check:
                s_check.add_assertion(Rn)
                s_check.add_assertion(Um)
                s_check.add_assertion(Eq_top)
                if s_check.solve():
                    print("unsafe n = {} m = {}".format(n,m))
                    model=s_check.get_model()
                    
                    print(model)
                    return
            A=And(Rn,Eq_top)
            Bf=Um
            C=interpolate(A,Bf)
            
            C_on_Xn=rename(C,X[n])
            C_on_Ym=rename(C,Y[n])
            
            if is_sat(And(Rn,Not(C_on_Xn))):
                pass
            if is_sat(And(Um,Not(C_on_Ym))):
                pass
            
            if n < N:
                C_on_Xn=rename(C,X[n])
                C_on_Xn=rename(C,X[n+1])
                T_between=trans(X[n],X[n+1])
                if not is_sat(And(C_on_Xn,T_between,Not(C_on_Xn1))):
                    print("safe(invaruant found)n={} m = {}").format(n,m)
                    return
            
            S=C_on_Xn
            sucess_maj= False
            for iter_i in range(max_maj):
                A_major=And(S,trans(X[n],Y[m]))
                
                if is_sat(And(A_major,Um)):
                    sucess_maj=False
                    break
                
                C2 = interpolate(A_major,Um)
                
                C2_on_Xn=rename(C2,X[n])
                if not is_sat(And(C2_on_Xn,Not(s))):
                    if not is_sat(And(S,Um)):
                        print("safe(majorant invariant s found)n={} m = {} on {} iteerations").format(n,m,iter_i+1)
                        sucess_maj=True
                        break
                    else:
                        sucess_maj=False
                        break
                
        print("unknown")                   
                        

#####

model_checking(['x','pc'], init1, trans1, error1, 50, 50)            

TypeError: model_checking() missing 1 required positional argument: 'max_maj'

### Exercício 7

 Como deve ter visto pela resposta do algoritmo anterior, o nosso programa é inseguro. Defina uma precondição mais forte de modo a que o programa seja seguro.

In [ ]:
# completar